In [1]:
import os
import glob
import pickle
from datetime import datetime
import time
import dotenv
import pandas as pd

import requests
import requests.auth

import praw

import openai

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# load secrets from .env into environment variables
dotenv.load_dotenv()

praw.__version__

'7.7.0'

See README.md
 - objective is to use OpenAI for named entity extraction to extract all the songs form [this reddit thread](https://www.reddit.com/r/AskReddit/comments/12viv4v/what_is_the_prettiest_song_you_ever_heard_in_your/) and make Spotify playlist
 - use Reddit PRAW API to download all the comments (get [Reddit API key](https://www.reddit.com/prefs/apps))
 - use OpenAI API with a prompt like, extract all the songs from this text to CSV get ([OpenAI API key](https://platform.openai.com/account/api-keys))
 - use Spotify API to make a playlist (get [Spotify API key](https://developer.spotify.com/documentation/web-api/tutorials/getting-started))
 - works, needed a lot of scrubbing, but about 1 day of work, wouldn't have been possible to do a 700-song playlist manually without a team of Mechanical Turks or something
 - If I wanted to go nuts, would process comments individually, save a file for each comment's extracted songs, would make it easier to track down what OpenAI gets wrong, have a resumable, retryable, repeatable process and 
 - Spotify playist is [here](https://open.spotify.com/playlist/08YFkbtTV6GBfNtjJ4PHDu?si=f4761d983ac84091) 
 
 needs a .env file per dot-env-template
 

In [24]:
# a thread 
submission = "12viv4v"

# minimum karma to process a reply 
minkarma = 5

# a prompt to apply to replies on the thread
# prefix = """Define an example CSV file output as follows: 
# "artist","song_title"
# "The Beatles","Yesterday"
# "Eagles","Hotel California"

# Extract all song titles and artists from the following input, and return a CSV file output of the artists and song titles you extract from the input. If there were no songs extracted from the input, return "no songs found". the input is:
# """

system_prompt="You will act as a research assistant finding all the songs and artists in a document and returning them a a CSV file"
assistant_prompt="""Define a CSV file output as follows: 
artist,song_title
"The Beatles","Yesterday"
"Eagles","Hotel California" """
user_prefix="Extract all song titles and artists from the following input, and return a CSV file output containing the artists and song titles you extract from the input. The header row should contain 'artist,song_title'. the input is:"
# an output file to accumulate all the responses
savefile = 'bronze.txt'


## Get all comments from a reddit posting

In [3]:
def getPraw():
    return praw.Reddit(user_agent="prettiest_song/0.001", 
                       client_id=os.getenv('CLIENT_ID'), 
                       client_secret=os.getenv('CLIENT_SECRET'))


def getAll(r, submissionId, verbose=True):
    submission = r.submission(submissionId)
    submission.comments.replace_more(limit=None)
    commentsList=submission.comments.list()
    return commentsList


In [4]:
print(datetime.now())
r = getPraw()
res = getAll(r, submission)
print(datetime.now())

print("retrieved ", len(res), 'comments')

2023-04-30 16:49:10.659259
2023-04-30 17:29:18.276437
retrieved  24839 comments


In [7]:
# we have a list of comment objects
# filter comments with at least some karma
res3 = [r for r in res if r.score >= minkarma]
print('filtered to ', len(res3), 'comments')
res3[0].body, res3[0].score

filtered to  2515 comments


('Gymnopédies - Erik Satie', 6925)

In [8]:
# save so we can reload it later without downloading

with open('reddit.pkl', 'wb') as f:
    pickle.dump(res3, f)
    
with open('reddit.pkl', 'rb') as f:
    res3 = pickle.load(f)
    

## Extract artists and song titles using OpenAI

In [9]:
# check lengths of posts
shorties = []
big_ones = []
for i in range(len(res3)):
    if len(res3[i].body) <3:
        print (i, res3[i].body)
        shorties.append(i)
    if len(res3[i].body) > 4096:
        print(i, len(res3[i].body))
        big_ones.append(i)
        

474 4162
1539 W


In [10]:
# avg length
sum([len(r.body) for r in res3]) / len(res3)

105.34075546719681

In [11]:
print (res3[big_ones[0]].body[:500])

Saturn by Sleeping at Last:
https://www.youtube.com/watch?v=dzNvk80XY9s

The version they did with Tim Fain is even more beautiful: 
https://www.youtube.com/watch?v=0nRpeAiur9Q

I'm not good at choosing one thing from a list of favorites as the best, so I've got about 30+ answers that are really a 30+ -way tie, and the one that I would consider as "prettiest" at any given moment is heavily influenced by my current mood. So, it could be any one of these from my "Heart Wrenchingly Beautiful" playl


In [28]:
# for each comment object we will extract the body 
# then submit as part of a prompt to chatgpt
print(datetime.now())

openai.api_key = os.getenv('OPENAI_API_KEY')

slist = res3.copy()

# to speed things we'll cumulate posts til we get to 100 posts or 5000 chars, whichever comes first
maxchars = 5000  # max tokens is 4096 but we'll limit each prompt to 5000 chars
nposts = 100

# make sure no single post > maxchars + prefix which breaks the logic below
for i in range(len(slist)):
    if len(slist[i].body) > maxchars + len(prefix):
        print ("truncated ", i)
        slist[i].body = slist[i].body[:maxchars + len(prefix)]
        
outdir = 'out'
logdir = 'logs'
# make sure out and logs are empty
for f in glob.glob('%s/*' % outdir):
    os.remove(f)
for f in glob.glob('%s/*' % logdir):
    os.remove(f)
count = 0
c = 0


while(slist):  # still comments to process
    prompt = ""
    reply_ids = []

    for _ in range(nposts):  # add up to 100 posts to the prompt
        if slist:
            if len(prompt) + len(slist[0].body) < maxchars:
                reply = slist.pop(0)
                reply_ids.append(reply.id)
                body = reply.body # in order, for better context 
#                 if len(body) <3:
#                     print (c, body)
                prompt += body
                prompt += " \n \n"
#                 c += 1            
            
    # retry loop, have received untrapped 502 error
    for _ in range(3):
        try:
            response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo-0301',
                messages=[{"role":"system", "content": system_prompt},
                          {"role":"assistant", "content": assistant_prompt},
                          {"role":"user", "content": user_prefix + prompt}
                         ],
                temperature=0,
            )
        except Exception as error:
            print("An exception occurred:", error)
            print("looping...")
            time.sleep(5)
            continue  # try again
        break   # exception not triggered

    with open("%s/%04d.csv" % (outdir, count), 'w') as outfile:
        outfile.write(response['choices'][0]['message']['content'])
    
    with open("%s/%04d.log" % (logdir, count), 'w') as logfile:
        logfile.write(str(reply_ids))
        logfile.write('\n\n===========\n\n')        
        logfile.write(prompt)
        logfile.write('\n\n===========\n\n')
        logfile.write(response['choices'][0]['message']['content'])
 
    count += 1
#     print(c)
    print('.', end='')

print()
print(datetime.now())



2023-04-30 17:42:33.984027
..........................................................
2023-04-30 17:54:30.747307


In [55]:
filelist = os.listdir(outdir)

output_df = None

for f in sorted(filelist):
    print(f)
    tempdf = pd.read_csv("%s/%s" % (outdir, f))
    colcount = len(tempdf.columns)
    if len(tempdf.columns) != 2:
        print('%s has %d columns' % (f, colcount))
        continue
    elif tempdf.columns[0] != "artist":
        print('%s 1st column is %s' % (f, tempdf.columns[0]))
        continue
    elif tempdf.columns[1] != "song_title":
        print('%s 2nd column is %s' % (f, tempdf.columns[1]))
        continue
    if output_df is not None:
        output_df = pd.concat([output_df, tempdf], axis=0)
    else:
        output_df = tempdf



0000.csv
0001.csv
0002.csv
0003.csv
0004.csv
0005.csv
0006.csv
0007.csv
0008.csv
0009.csv
0010.csv
0011.csv
0012.csv
0013.csv
0014.csv
0015.csv
0016.csv
0017.csv
0018.csv
0019.csv
0020.csv
0021.csv
0022.csv
0023.csv
0024.csv
0025.csv
0026.csv
0027.csv
0028.csv
0029.csv
0030.csv
0031.csv
0032.csv
0033.csv
0034.csv
0035.csv
0036.csv
0037.csv
0038.csv
0039.csv
0040.csv
0041.csv
0042.csv
0043.csv
0044.csv
0045.csv
0046.csv
0047.csv
0048.csv
0049.csv
0050.csv
0051.csv
0052.csv
0053.csv
0054.csv
0055.csv
0056.csv
0057.csv


In [61]:
# save bronze
output_df.to_csv(savefile, index=False)


In [63]:
# may have to tweak the file to get it to load
df = pd.read_csv(savefile) \
    .drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "song_title"]) \
    .reset_index(drop=True)

df.to_csv('silver.csv', index=False)

print(len(df))
# tweak further to get to gold.csv

df


1136


,artist,song_title
0,311,Amber
1,A Perfect Circle,Blue
2,A Perfect Circle,Gravity
3,A Perfect Circle,Judith
4,A Perfect Circle,Orestes
...,...,...
1131,beabadoobee,glue song
1132,delibes,Flower duet
1133,paper kites,Bloom
1134,unknown,unknown


In [77]:
df.groupby('artist') \
    .count() \
    .reset_index() \
    .sort_values('song_title', ascending=False) \
    .head(10)



,artist,song_title
233,Jimi Hendrix,62
445,The Beatles,31
380,Radiohead,17
412,Simon & Garfunkel,17
413,Simon and Garfunkel,16
455,The Cure,14
146,Enya,14
29,Aphex Twin,13
232,Jim Croce,13
436,Taylor Swift,12


In [75]:
df = df.drop(df.loc[df['artist']=='Unknown'].index)
df = df.drop(df.loc[df['artist']=='unknown'].index)

In [81]:
df.groupby('song_title') \
    .count() \
    .reset_index() \
    .sort_values('artist', ascending=False) \
    .head(10)


,song_title,artist
589,Nothing Compares 2 U,6
302,Hallelujah,6
101,Blackbird,5
341,Hurt,4
540,Moon River,4
427,La Vie En Rose,4
240,Falling Slowly,4
200,Dream A Little Dream Of Me,4
207,Dreams,3
730,Song to the Siren,3


In [80]:
df = df.drop(df.loc[df['song_title']=='Unknown'].index)
df = df.drop(df.loc[df['song_title']=='unknown'].index)

In [82]:
len(df)

1087

In [83]:
df.to_csv('silver.csv', index=False)


## Load into a Spotify playlist


In [84]:
client_credentials_manager = SpotifyClientCredentials(client_id=os.getenv('SPOTIFY_CLIENT_ID'), 
                                                      client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
                                                      )

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [85]:
# check artists
df = pd.read_csv("silver.csv")
df.drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "song_title"])

dedupe = {}
fail_list = []

for index, artist, title in df.itertuples():
    if artist in dedupe:
        continue
    dedupe[artist]=1
    query_str = 'artist:%s' % (artist)
    artist_results = sp.search(q=query_str, type='artist', limit=3, offset=0, market='US')
    artist_names = [artist['name'] for artist in artist_results['artists']['items']]
    if artist_names:
        if artist.lower() != artist_names[0].lower():
            print(artist, artist_names)
    else:
        fail_list.append((artist, title))
        print("not found:", artist, "-", title)

# then clean up manually as appropriate

A-Ha ['Daryl Hall & John Oates', 'a-ha', 'half•alive']
Al Stewart ['Alexander Stewart', 'Al Stewart', 'Alec Lee-Stewart']
Alan Parsons Project ['The Alan Parsons Project', 'The Alan Parsons Symphonic Project', 'The Alan Parsons Tribute Project']
not found: Allison Kraus - When You Say Nothing at All
not found: Allison Kraus and Brad Paisley - Whiskey Lullaby
America ['The All-American Rejects', 'America', 'American Authors']
not found: Artic Monkeys - The Lovecats
not found: Avril Altdelay - 05quVI7Ioqc
Babyface ['Babyface Ray', 'Babyface', 'BabyFaceWood']
Bach ['Johann Sebastian Bach', 'Bachman-Turner Overdrive', 'Tal Bachman']
Barbara ['Ana Bárbara', 'Barbara Mandrell', 'Barbara Mason']
not found: Barry De Vorzon and Perry Botkin Jr. - Nadia's Theme
Beatles ['The Beatles', 'The Beatles Complete On Ukulele', 'The New Beatles']
Berlin ['Berliner Philharmoniker', 'Berlin', 'Konzerthaus Kammerorchester Berlin']
Beyonce ['Beyoncé', 'Beyonce Smith', 'Mc Beyonce']
not found: Bill Withers & 

In [86]:
# check songs

df = pd.read_csv("silver.csv")
df.drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "song_title"])

dedupe = {}
mylist = []
fail_list = []
artist_list, track_list, uri_list, album_list = [], [], [], []
orig_artist, orig_track = [], []

for index, artist, title in df.itertuples():
    query_str = 'artist:%s track:%s' % (artist, title)
    track_results = sp.search(q=query_str, type='track', limit=1, offset=0, market='US')
    results = track_results['tracks']['items']
    
    if results:
        r = results[0]
        # failsafe to never put same track twice
        if dedupe.get(r['id']):
            continue
        dedupe[r['id']]=True
        if title.lower() != r['name'].lower():
            print ("%s|%s : %s|%s" % (artist, title, r['artists'][0]['name'], r['name']))
        uri_list.append(r['uri'])
        artist_list.append(r['artists'][0]['name'])
        track_list.append(r['name'])
        album_list.append(r['album']['name'])
        orig_artist.append(artist)
        orig_track.append(title)
#         print('  ',
#               r['artists'][0]['name'],'|',
#               r['name'], '|',
#               r['album']['name'],'|',
#               r['album']['release_date'],'|',
#               r['popularity'])
    else:
        fail_list.append((artist, title))
        print("not found:", artist, "-", title)

not found: A Perfect Circle - The Nurse who Loved me (cover)
A Perfect Circle|Three Libras : A Perfect Circle|3 Libras
not found: A Perfect Circle - Wings for Marie pt 1&2
Alice in Chains|Rain When I Die : Alice In Chains|Rain When I Die (2022 Remaster)
not found: Allison Kraus - When You Say Nothing at All
not found: Allison Kraus and Brad Paisley - Whiskey Lullaby
Andy Williams|Moon River : Andy Williams|Moon River (From "Breakfast at Tiffany's")
not found: Animal Collective - Animal Collective - banshee beat
not found: Aphex Twin - Aphex Twin - Fingerbib
not found: Aphex Twin - Avrl 14
not found: Aphex Twin - I Z Us
not found: Aphex Twin - Lichen
Aphex Twin|Rhubarb : Aphex Twin|Donkey Rhubarb
not found: Aphex Twin - Stone in Focus
not found: Aphex Twin - Untitled #17
not found: Artic Monkeys - The Lovecats
Arvo Pärt|Spiegel im Spiegel : Arvo Pärt|Spiegel im Spiegel - Version for Violin and Piano
not found: Aurora - Any song by Aurora
not found: Avril Altdelay - 05quVI7Ioqc
Babyface|

Grimes|Symphonia IX : Grimes|Symphonia IX (My Wait Is U)
not found: Guitars and Dragons - Concerning Hobbits
Gustav Holst|Jupiter : Gustav Holst|The Planets, Op. 32: 4. Jupiter, the Bringer of Jollity
Hans Zimmer|Chevaliers de Sangreal : Hans Zimmer|Chevaliers De Sangreal - From The Da Vinci Code Original Motion Picture Soundtrack
not found: Hans Zimmer - Interstellar Intro
Hans Zimmer|Tennessee : Hans Zimmer|Tennessee (from "Pearl Harbor")
Hans Zimmer|This Land : Hans Zimmer|This Land - From "The Lion King"/Score
not found: Hayde Bluegrass Orchestra version - All My Tears
Heart|Ann : Heart|Dreamboat Annie
Heart|Nancy : Hellbound Hearts|Nancy's House
not found: Henri Mancini - Moon River
Henry Mancini|Moon River : Henry Mancini|Moon River(Vocal Audrey Hepburn)
Honeydrippers|Sea Of Love : The Honeydrippers|Sea of Love - 2006 Remaster
Howard Hanson|Symphony 2 : Howard Hanson|Symphony No. 2, Op. 30, "Romantic": I. Adagio - Allegro moderato
Hozier|Cherry Wine : Hozier|Cherry Wine - Live
no

not found: Leonard Cohen - Nothing Compares 2 U
Les Miserables|I dreamed a dream : Lesley Garrett|Schönberg, Claude-Michel: I Dreamed a Dream (from "Les Misérables")
Lisa Gerrard|Sanvean : Lisa Gerrard|Sanvean - I Am Your Shadow
not found: Little Earthquakes - Bells for Her
Loggins & Messina|House On Pooh Corner : Loggins & Messina|House at Pooh Corner - Live
not found: Lord Huron - Blue Ridge Mountains
Lord Huron|Ghost On The Shore : Lord Huron|The Ghost on the Shore
Lord Huron|I lied : Lord Huron|I Lied (with Allison Ponthier)
Loreena McKennitt|Never-Ending Road : Loreena McKennitt|Never-Ending Road (Amhrán Duit)
not found: Louis/Ella - Blackbird
Ludovico Einaudi|Fly : Ludovico Einaudi|Film Music, Pt. 1: III. Fly
Ludwig van Beethoven|Ode to Joy : Ludwig van Beethoven|Beethoven: 'Ode To Joy' From Symphony No. 9 In D Minor 'Choral', Op. 125
Ludwig van Beethoven|all works : Ludwig van Beethoven|11 Bagatelles, Op. 119: 1. Allegretto
Léo Delibes|The Flower Duet : Léo Delibes|The Flower Du

The Beatles|And I love her : The Beatles|And I Love Her - Remastered 2009
not found: The Beatles - Astral Weeks
The Beatles|Blackbird : The Beatles|Blackbird - Remastered 2009
not found: The Beatles - Claire de Lune
The Beatles|Dear Prudence : The Beatles|Dear Prudence - Remastered 2009
The Beatles|Golden Slumbers : The Beatles|Golden Slumbers - Remastered 2009
The Beatles|Golden Slumbers/Carry That Weight : The Beatles|Golden Slumbers / Carry That Weight - Takes 1-3 / Medley
The Beatles|Golden Slumbers/Carry That Weight/The End : The Beatles|The Long One - Comprising of ‘You Never Give Me Your Money’, ’Sun King’/’Mean Mr Mustard’, ‘Her Majesty’, ‘Polythene Pam’/’She Came In Through The Bathroom Window’, ’Golden Slumbers’/ ’Carry That Weight’, ’The End’
The Beatles|Here Comes The Sun : The Beatles|Here Comes The Sun - Remastered 2009
The Beatles|Here, There, and Everywhere : The Beatles|Here, There And Everywhere - Remastered 2009
The Beatles|I Wanna Hold Your Hand : The London Orchest

In [87]:
gold_df = pd.DataFrame({'input_artist': orig_artist,
                        'artist': artist_list,
                        'input_track': orig_track,
                        'track': track_list,
                        'album': album_list,
                        'uri': uri_list})
gold_df

,input_artist,artist,input_track,track,album,uri
0,311,311,Amber,Amber,Greatest Hits '93 - '03,spotify:track:51UtgWS4z1eMPuLQOzPtNH
1,A Perfect Circle,A Perfect Circle,Blue,Blue,Thirteenth Step,spotify:track:6tgTTBaIf0tO6lvDhoXfMg
2,A Perfect Circle,A Perfect Circle,Gravity,Gravity,Thirteenth Step,spotify:track:1CO4BB8CaiQggtJ0R6GwGt
3,A Perfect Circle,A Perfect Circle,Judith,Judith,Mer De Noms,spotify:track:5KDNFlHAdDJ84fhK27c35X
4,A Perfect Circle,A Perfect Circle,Orestes,Orestes,Mer De Noms,spotify:track:6YUKAH1icwkA3U7fVp3amo
...,...,...,...,...,...,...
795,Yo La Tengo,Yo La Tengo,My Little Corner of the World,My Little Corner of the World,I Can Hear The Heart Beating As One,spotify:track:2bY66Hf5NbHJ8Ai8eNmJHG
796,alt-J,alt-J,Dissolve Me,Dissolve Me,An Awesome Wave,spotify:track:2Dv7PTwSoB17f3VFDIKw8m
797,beabadoobee,beabadoobee,glue song,Glue Song,Glue Song,spotify:track:3iBgrkexCzVuPy4O9vx7Mf
798,delibes,Léo Delibes,Flower duet,Flower Duet,Classical Music 101,spotify:track:5K8jqeLAxZIqHR6e5w5so1


In [88]:
with pd.option_context("display.max_rows", 999):
    display(gold_df.loc[gold_df['input_artist'].str.lower() != gold_df['artist'].str.lower()])

,input_artist,artist,input_track,track,album,uri
15,Alan Parsons Project,The Alan Parsons Project,Old and Wise,Old and Wise,Eye In The Sky (Expanded Edition),spotify:track:5Jt2AQv1c3RUF5ENtAYF1i
16,Alan Parsons Project,The Alan Parsons Project,Silence and I,Silence and I,Eye In The Sky (Expanded Edition),spotify:track:1MXd1awM0A7T7FxUPKAujf
17,Alan Parsons Project,The Alan Parsons Project,Time,Time,The Turn Of A Friendly Card (Expanded Edition),spotify:track:48yJZwYYDZX5GKFND7wDfC
21,Alison Krauss,Alison Krauss & Union Station,New Favorite,New Favorite,New Favorite,spotify:track:4iVeKORDsSUxkLzmme2bbL
28,America,American Folk Channel,The Boxer,The Boxer,American Country Folk for the Trucks 2,spotify:track:6jLXjtanB0Yp82aAV7VGsN
53,Bach,Johann Sebastian Bach,Air on the G String,"Orchestral Suite No. 3 in D Major, BWV 1068: I...",Sleep for Baby,spotify:track:77GRzY2dy2IsSpSssSHPjq
62,Beatles,The Beatles,In My Life,In My Life - Remastered 2009,Rubber Soul (Remastered),spotify:track:3KfbEIOC7YIv90FIfNSZpo
67,Beyonce,Beyoncé,Plastic off the sofa,PLASTIC OFF THE SOFA,RENAISSANCE,spotify:track:6ufcuVInt0ocHrUimDjGlb
77,Bjork,Björk,Hyperballad,Hyperballad,Post,spotify:track:4z1fNs2B7KndCsvyPgrhq5
78,Bjork,Björk,Unison,Unison,Vespertine,spotify:track:1t9us2hggVqmnyWz7uG54y


In [ ]:
with pd.option_context("display.max_rows", 999):
    display(gold_df.loc[gold_df['input_track'].str.lower() != gold_df['track'].str.lower()])

In [ ]:
gold_df.iloc[29]

In [89]:
# these are songs that look like covers or otherwise not the expected response from spotify search 
# (which is a bit wonky, doesn't like quotes and such)

bad_lookups = [
    28, 79, 110, 286, 675, 693, 696, 697, 698, 699, 721, 726, 754
]

for i in bad_lookups:
    print(gold_df.iloc[i])
    
# add manually, plus 'not found'


input_artist                                   America
artist                           American Folk Channel
input_track                                  The Boxer
track                                        The Boxer
album           American Country Folk for the Trucks 2
uri               spotify:track:6jLXjtanB0Yp82aAV7VGsN
Name: 28, dtype: object
input_artist                                                Bjork
artist                                                 Björkliden
input_track                                                  Yoga
track                                                  Yoga Nidra
album           Meditation Spa yoga healing therapy relaxation...
uri                          spotify:track:3AdIPVFDkqSSvBjKFcN0mr
Name: 79, dtype: object
input_artist                                    Cher
artist                                Cherie Nichole
input_track                               Moonstruck
track                                     Moonstruck
album         

In [90]:
gold_df = gold_df.drop(
    axis='index',
    labels=bad_lookups)

gold_df[['artist', 'track']].to_csv('gold.csv', index=False)

with pd.option_context("display.max_rows", 999):
    display(gold_df)

,input_artist,artist,input_track,track,album,uri
0,311,311,Amber,Amber,Greatest Hits '93 - '03,spotify:track:51UtgWS4z1eMPuLQOzPtNH
1,A Perfect Circle,A Perfect Circle,Blue,Blue,Thirteenth Step,spotify:track:6tgTTBaIf0tO6lvDhoXfMg
2,A Perfect Circle,A Perfect Circle,Gravity,Gravity,Thirteenth Step,spotify:track:1CO4BB8CaiQggtJ0R6GwGt
3,A Perfect Circle,A Perfect Circle,Judith,Judith,Mer De Noms,spotify:track:5KDNFlHAdDJ84fhK27c35X
4,A Perfect Circle,A Perfect Circle,Orestes,Orestes,Mer De Noms,spotify:track:6YUKAH1icwkA3U7fVp3amo
5,A Perfect Circle,A Perfect Circle,The Noose,The Noose,Thirteenth Step,spotify:track:6lvNLD1XRU5paMwWH0RGRI
6,A Perfect Circle,A Perfect Circle,Three Libras,3 Libras,Mer De Noms,spotify:track:5kHkaBN8OEQlmXfQkACxSt
7,A-Ha,a-ha,Take on Me,Take on Me,Hunting High and Low,spotify:track:2WfaOiMkCvy7F5fcp2zZ8L
8,ABBA,ABBA,Fernando,Fernando,Arrival,spotify:track:4BM8yJ0PzBi2ZewpMTOxtx
9,ABBA,ABBA,One of us,One Of Us,The Essential Collection,spotify:track:6zgtBUEkAfilJ2YEOvNexR


In [91]:
# get playlist id
# first create a playlist in UI to load songs
playlists = sp.user_playlists(os.getenv('SPOTIFY_USERNAME'))
while playlists:
    for i, playlist in enumerate(playlists['items']):
        if playlist['name'] != 'Reddit Prettiest Songs':
            continue
        print(playlist['id'])
        playlist_id = playlist['id']
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

08YFkbtTV6GBfNtjJ4PHDu
   1 spotify:playlist:08YFkbtTV6GBfNtjJ4PHDu Reddit Prettiest Songs


In [100]:
# must follow an oauth workflow to write a playlist in Spotify
# running this cell should request a spotify login and then redirect to an url
# paste whole url with id into form to authenticate

scope = "playlist-modify-public"

sp = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth(scope=scope,
                                                       client_id=os.getenv('SPOTIFY_CLIENT_ID'),
                                                       client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
                                                       redirect_uri="https://druce.ai"
                                                      ))

In [ ]:
addlist = gold_df['uri'].to_list()
print (len(addlist))

while(addlist):
    sp.user_playlist_add_tracks(os.getenv('SPOTIFY_USERNAME'), 
                                playlist_id=playlist_id, 
                                tracks=addlist[-100:])
    addlist = addlist[:-100]
    print("added items, remaining ", len(addlist))


In [ ]:
# manually add the ones that weren't found for some reason


In [92]:
# can run again and add any new tracks, either because OpenAI is a bit random, or new replies in thread
results = sp.user_playlist(os.getenv('SPOTIFY_USERNAME'), playlist_id,
                                fields='tracks,next,name')
tracks = results['tracks']

playlist_dict_by_uri = {}
playlist_dict_by_str = {}

while True:
    for track_item in tracks['items']:
        track_dict = track_item['track']
        track_str = track_dict['artists'][0]['name']  + ' | ' + track_dict['name'][:15]
        uri = track_dict['uri']
        if track_str in playlist_dict_by_str:
            print(track_str)
        playlist_dict_by_str[track_str] = uri
        playlist_dict_by_uri[uri] = track_str
    # check if there are more pages
    if tracks['next']:
        tracks = sp.next(tracks)
    else:
        break

print (len(list(playlist_dict_by_str.keys())))
print (len(list(playlist_dict_by_uri.keys())))


878
878


In [93]:
gold_dict_by_uri = {}
gold_dict_by_str = {}
addlist = []
c = 0
for i, artist, track, uri in gold_df[['artist', 'track', 'uri']].itertuples():
    # print(artist, track, uri)
    track_str = artist + ' | ' + track[:15]
    if track_str not in playlist_dict_by_str:
        addlist.append([artist, track, uri])
        print(artist, track, uri)
    gold_dict_by_uri[uri]=track_str
    gold_dict_by_str['track_str']= uri
#     if track_str not in playlist_dict_by_str:
#         c += 1
#         print (c, track_str)
        
print(len(gold_dict_by_str.items()))
print(len(gold_dict_by_uri.items()))

A Perfect Circle Gravity spotify:track:1CO4BB8CaiQggtJ0R6GwGt
a-ha Take on Me spotify:track:2WfaOiMkCvy7F5fcp2zZ8L
The Alan Parsons Project Old and Wise spotify:track:5Jt2AQv1c3RUF5ENtAYF1i
The Alan Parsons Project Silence and I spotify:track:1MXd1awM0A7T7FxUPKAujf
Alison Krauss & Union Station New Favorite spotify:track:4iVeKORDsSUxkLzmme2bbL
Alter Bridge Blackbird spotify:track:6pRPoKyrZ2VNlgpvU6wwNB
Amy Winehouse Love Is A Losing Game spotify:track:3uliGwmB52ZA7brgpZMzyH
Andrew Bird Beyond the Valley of the Three White Horses spotify:track:1DP0urxeM1qc2O9ky2vrDM
Andy Williams Moon River (From "Breakfast at Tiffany's") spotify:track:24AIahNHzBxm9S12peXbnG
Anne Murray Danny's Song spotify:track:5lQxMlCMTzCj3j4vyzdl8T
Antônio Carlos Jobim The Girl From Ipanema spotify:track:5N6pQ2vYtd3Rb9w7LC8PZ9
Aphex Twin Alberto Balsalm spotify:track:6gbmylJ7sB7NFfMfTQHosf
Aphex Twin Xtal spotify:track:7o2AeQZzfCERsRmOM86EcB
Archspire Reverie on the Onyx spotify:track:0Ond2khar20tMNRFf4pQR0
Barbara 

In [96]:
addlist

[['A Perfect Circle', 'Gravity', 'spotify:track:1CO4BB8CaiQggtJ0R6GwGt'],
 ['a-ha', 'Take on Me', 'spotify:track:2WfaOiMkCvy7F5fcp2zZ8L'],
 ['The Alan Parsons Project',
  'Old and Wise',
  'spotify:track:5Jt2AQv1c3RUF5ENtAYF1i'],
 ['The Alan Parsons Project',
  'Silence and I',
  'spotify:track:1MXd1awM0A7T7FxUPKAujf'],
 ['Alison Krauss & Union Station',
  'New Favorite',
  'spotify:track:4iVeKORDsSUxkLzmme2bbL'],
 ['Alter Bridge', 'Blackbird', 'spotify:track:6pRPoKyrZ2VNlgpvU6wwNB'],
 ['Amy Winehouse',
  'Love Is A Losing Game',
  'spotify:track:3uliGwmB52ZA7brgpZMzyH'],
 ['Andrew Bird',
  'Beyond the Valley of the Three White Horses',
  'spotify:track:1DP0urxeM1qc2O9ky2vrDM'],
 ['Andy Williams',
  'Moon River (From "Breakfast at Tiffany\'s")',
  'spotify:track:24AIahNHzBxm9S12peXbnG'],
 ['Anne Murray', "Danny's Song", 'spotify:track:5lQxMlCMTzCj3j4vyzdl8T'],
 ['Antônio Carlos Jobim',
  'The Girl From Ipanema',
  'spotify:track:5N6pQ2vYtd3Rb9w7LC8PZ9'],
 ['Aphex Twin', 'Alberto Balsal

In [97]:
# clean up manually, covers, eagles of death metal etc
addlist = [
    ['A Perfect Circle', 'Gravity', 'spotify:track:1CO4BB8CaiQggtJ0R6GwGt'],
 ['The Alan Parsons Project',
  'Old and Wise',
  'spotify:track:5Jt2AQv1c3RUF5ENtAYF1i'],
 ['The Alan Parsons Project',
  'Silence and I',
  'spotify:track:1MXd1awM0A7T7FxUPKAujf'],
 ['Alter Bridge', 'Blackbird', 'spotify:track:6pRPoKyrZ2VNlgpvU6wwNB'],
 ['Amy Winehouse',
  'Love Is A Losing Game',
  'spotify:track:3uliGwmB52ZA7brgpZMzyH'],
 ['Andy Williams',
  'Moon River (From "Breakfast at Tiffany\'s")',
  'spotify:track:24AIahNHzBxm9S12peXbnG'],
 ['Anne Murray', "Danny's Song", 'spotify:track:5lQxMlCMTzCj3j4vyzdl8T'],
 ['Antônio Carlos Jobim',
  'The Girl From Ipanema',
  'spotify:track:5N6pQ2vYtd3Rb9w7LC8PZ9'],
 ['Aphex Twin', 'Alberto Balsalm', 'spotify:track:6gbmylJ7sB7NFfMfTQHosf'],
 ['Aphex Twin', 'Xtal', 'spotify:track:7o2AeQZzfCERsRmOM86EcB'],
 ['Archspire', 'Reverie on the Onyx', 'spotify:track:0Ond2khar20tMNRFf4pQR0'],
 ['Barbara',
  "Ma plus belle histoire d'amour",
  'spotify:track:0qBVET4VkHsQAoboWlQ2pJ'],
 ['Berlin',
  'Take My Breath Away - Love Theme from "Top Gun"',
  'spotify:track:15MJ5NThPjj6xhPcts8MiY'],
 ['Bon Iver', '33 “GOD”', 'spotify:track:5sWF1EZGm727f3b620vkbC'],
 ['Bon Iver', '8 (circle)', 'spotify:track:16WkFbQRW2BhvMUwuIaIsx'],
 ['Bon Iver', 'Hey, Ma', 'spotify:track:0RstfX9nRY1Lfuy1808MoT'],
 ['Bon Iver', 'Michicant', 'spotify:track:43d83eCd7GzmVOKsLwV3QN'],
 ['Bon Iver', 'Minnesota, WI', 'spotify:track:6dmqnKIOvBfqjP317AT2tT'],
 ['Bon Iver', 'Perth', 'spotify:track:5sYAPWzsEk8pd1OFhcQ67T'],
 ['Chris Isaak', 'Wicked Game', 'spotify:track:7w5AOd6HrDIHewHfpABEss'],
 ['Claude Debussy',
  '2 Arabesques, L. 66: No. 1 in E Major',
  'spotify:track:53zrZribB4xc2uRzZXWKCT'],
 ['Colin Hay', 'Beautiful World', 'spotify:track:7E1gZbAh7yzpXH6SUsN2Rr'],
 ['Crosby, Stills & Nash',
  'Helplessly Hoping - 2005 Remaster',
  'spotify:track:1UKobFsdqNXQb8OthimCKe'],
 ['Daniela Andrade', 'La Vie En Rose', 'spotify:track:0VxocBntP1XTZRsR9ZURPS'],
 ['David Gilmour',
  'Time - Live In Gdansk',
  'spotify:track:0dDvdSqu5LcgxZmBBguuuF'],
 ['Death Cab for Cutie',
  'Brothers on a Hotel Bed',
  'spotify:track:2MASSWjF0hSSj6nj9OftoR'],
 ['Death Cab for Cutie',
  'Grapevine Fires',
  'spotify:track:1x3bxSSANgIxkoyGldhmdX'],
 ['Dolly Parton', 'Jolene', 'spotify:track:2SpEHTbUuebeLkgs9QB7Ue'],
 ['Don McLean', 'Vincent', 'spotify:track:0VNzEY1G4GLqcNx5qaaTl6'],
 ['Antonín Dvořák',
  'Dvorák / Transc. Lenaerts: Rusalka, Op. 114, Act 1: Song to the Moon',
  'spotify:track:5nOSt81jKAUYZpTdjcfEOA'],
 ['Elton John',
  'Captain Fantastic And The Brown Dirt Cowboy',
  'spotify:track:6dix9mJZambkCkDOlXQTwG'],
 ['Elton John',
  'Mona Lisas And Mad Hatters',
  'spotify:track:5XY9FK0mB0BS0u8YB2ehN9'],
 ['Elvis Presley',
  'Unchained Melody - Live at Ann Arbor, MI',
  'spotify:track:0OavtQSojULqejmC4Qbstr'],
 ['Enya', 'Afer Ventus', 'spotify:track:7maBYPeIpnx2hjnc9zqV03'],
 ['Enya', 'Smaointe', 'spotify:track:3Usvxmk9Skhn7zb91dNA0z'],
 ['Fleet Foxes',
  'Blue Ridge Mountains',
  'spotify:track:7bzzC7QIfflM9eEj2aqaYX'],
 ['Fleetwood Mac',
  'You Make Loving Fun - 2004 Remaster',
  'spotify:track:28H3wOEKsaEa8NBEYCLl4j'],
 ['Foo Fighters', 'Everlong', 'spotify:track:5UWwZ5lm5PKu6eKsHAGxOk'],
 ['Peter Gabriel',
  'In Your Eyes - 2012 Remaster',
  'spotify:track:4qN7nSAVTjPfOOz0wkRQpM'],
 ['Glen Hansard',
  "When Your Mind's Made Up",
  'spotify:track:1aAfiBF8ftfkONfzc0GTWz'],
 ['Gordon Lightfoot', 'Don Quixote', 'spotify:track:6XFx5TPLE3lCBlS68Ka7Qh'],
 ['Gordon Lightfoot',
  'Early Morning Rain',
  'spotify:track:5M1gX3DQvgc7GI6dBWX9FL'],
 ['Gordon Lightfoot',
  "Old Dan's Records",
  'spotify:track:1WRC4Szy06yqktnqaEQ4kv'],
 ['Gustavo Santaolalla',
  'The Last of Us',
  'spotify:track:285ieonEuLkll3zknYK2TY'],
 ['Hans Zimmer',
  'Tennessee (from "Pearl Harbor")',
  'spotify:track:6r6hxq0yIGvbTpXvno4id7'],
 ['Hozier', 'Shrike', 'spotify:track:083PU4XnYAOMAGVruma3vb'],
 ['Hozier', 'Swan Upon Leda', 'spotify:track:3JtVtwLYCy9WXcHLQyzRnl'],
 ['Huey Lewis & The News',
  'The Power Of Love',
  'spotify:track:2olVm1lHicpveMAo4AUDRB'],
 ['INXS', 'Not Enough Time', 'spotify:track:4nwgKaHg9TJahiSSmt3X1x'],
 ['Jackson Browne',
  'Running on Empty',
  'spotify:track:6aqm56xP40foYBBtAWWrnY'],
 ['Jim Croce',
  'The Hard Way Every Time',
  'spotify:track:7l71GmbFrpk142d80nol23'],
 ['Jimi Hendrix',
  'The Wind Cries Mary',
  'spotify:track:5H6Jp0syB5yEPk7SWYdlmk'],
 ['John Denver',
  'Rocky Mountain High',
  'spotify:track:1ne9wOtDF2jM6Cm8WBkaER'],
 ['John Prine',
  'I Remember Everything',
  'spotify:track:5jYq5kj7kt6Iokzv7XeZhZ'],
 ['John Waite', 'Missing You', 'spotify:track:1Qrdlkgg9I4J7r3P4kZNwr'],
 ['Johnny Cash', 'The Mercy Seat', 'spotify:track:0Ttx8uPwS2aHavnWOTyjeS'],
 ['Johnny Tillotson', 'Dreamy Eyes', 'spotify:track:7CiLTJANslRNGigDDDCB01'],
 ['Joni Mitchell Tribute by Strings Attached',
  'Blue Hotel Room',
  'spotify:track:1OpzCsETUDNjY4noJRjAtP'],
 ['Joni Mitchell',
  'Mitchell: Urge for Going (Instrumental Arrangement of the B-Side Track of the Joni Mitchell Single "You Turn Me on I\'m a Radio")',
  'spotify:track:1I1u9aTdxxQ7SDLgBB3V7b'],
 ['Josh Groban',
  'Empty Chairs at Empty Tables (from "Les Misérables")',
  'spotify:track:3yGmmiwMiHFytWRS2sbfMM'],
 ['Joy Division',
  'Shadowplay - 2007 Remaster',
  'spotify:track:4ZuC5MfGjRQs3pZtPxqMYP'],
 ['Judy Collins',
  'Song for Judith (Open the Door)',
  'spotify:track:7wliemSUQ2bYbNwMC2UmxZ'],
 ['Jóhann Jóhannsson', 'Arrival', 'spotify:track:4KBwrmtoKe5QjfW2RhhpOf'],
 ['Jónsi', 'Simple Gifts', 'spotify:track:1VtcOgkXF8qktP4DY3KzYd'],
 ['Lady Gaga', 'La Vie En Rose', 'spotify:track:3WKnfkgwrARwElktHSU5Ik'],
 ['Laura & Anton', 'La Vie en Rose', 'spotify:track:2hTw1PCV8R7ho29XT6nmLv'],
 ['Lord Huron',
  'Lost in Time and Space',
  'spotify:track:6BqmgqzgKNxPszu1I6gOpn'],
 ['Lord Huron', 'Love Like Ghosts', 'spotify:track:4UGoW08gaqIEWNTam1UNen'],
 ['Ludovico Einaudi', 'Divenire', 'spotify:track:6KpO4neDXxGdiEgsivZieY'],
 ['Ludovico Einaudi',
  'Film Music, Pt. 1: III. Fly',
  'spotify:track:38tAI7h6C7x8LtCdja6bHV'],
 ['Ludwig van Beethoven',
  '11 Bagatelles, Op. 119: 1. Allegretto',
  'spotify:track:6fj6PYJHAHBR7ZlLygQlgH'],
 ['Max Richter', "Vladimir's Blues", 'spotify:track:6a7vpKrVXdAyMItaQI9ZYS'],
 ['Ministry', 'Jesus Built My Hotrod', 'spotify:track:0BX3ysoHJvxmLEhPMAfb2z'],
 ['My Chemical Romance', 'Cancer', 'spotify:track:0GgN4MhR5GKn5IcKN0e0rG'],
 ['Nina Chuba', 'Who Hurt You', 'spotify:track:3noZEEUGNUNEZO9LwIs5sF'],
 ['Nobuo Uematsu',
  'To Zanarkand (Final Fantasy X)',
  'spotify:track:6lROBBAsACJMNA30K8tnuv'],
 ['Oceanlab',
  'Satellite - Radio Edit',
  'spotify:track:4iYnJlYgMqoPUBBfQpQYhJ'],
 ['Otis Redding', 'Direct Me', 'spotify:track:7oxX06wBXOJdGOEcMT0Svy'],
 ['Patsy Cline',
  "She's Got You - Single Version",
  'spotify:track:4H1POasJbrLNaQirsIH29e'],
 ['Patsy Cline',
  'Walking After Midnight',
  'spotify:track:7no4JBfskmG4Qz1hZATCqg'],
 ['Paul Mauriat', 'Love Is Blue', 'spotify:track:1PlZt5i69FgDl45KbHlLzr'],
 ['Penguin Cafe Orchestra',
  "The Sound Of Someone You Love Who's Going Away And It Doesn't Matter - 2008 Digital Remaster",
  'spotify:track:5mhl1LKUCh4mOdPwh0WwMM'],
 ['Peter Gabriel',
  'Mercy Street - 2012 Remaster',
  'spotify:track:5QQBBmD91ciW9cSlLyjVZP'],
 ['Peter Gabriel',
  'A Wonderful Day In A One-Way World',
  'spotify:track:1pakjRd0PcZBpe4p0SkHRY'],
 ['Peter Gabriel', 'Solsbury Hill', 'spotify:track:1CM1wOqD2AIjt2MWd31LV2'],
 ['Plants and Animals', 'Bye Bye Bye', 'spotify:track:3DTCoeWvtjekbwRr2gQzJJ'],
 ['Porcupine Tree', 'Lazarus', 'spotify:track:0OtXJQBLb7wPXL2XyN2cFW'],
 ['Portishead', 'Glory Box', 'spotify:track:3Ty7OTBNSigGEpeW2PqcsC'],
 ['Giacomo Puccini',
  "Tosca, S. 69, Act II: Vissi d'arte",
  'spotify:track:4rtYqCcX46D2BzupEtlD4u'],
 ['Queen', 'The Show Must Go On', 'spotify:track:3m86qx7aIeqFcYcQ1UXUvP'],
 ['Ray LaMontagne',
  'Hold You in My Arms',
  'spotify:track:1U1GtaJeYwGsPgqV9Me5bb'],
 ['Ray LaMontagne', 'Trouble', 'spotify:track:5aGkIyzLVYGuLIHkxpNAFF'],
 ['Roberta Flack',
  'Killing Me Softly With His Song',
  'spotify:track:3gsCAGsWr6pUm1Vy7CPPob'],
 ['Roy Orbison', 'Crying', 'spotify:track:6eLL7QTdMWdhhG4i3jHDR9'],
 ['Selena Gomez & The Scene',
  'Winter Wonderland',
  'spotify:track:7shn2DNJ4WDGNyHrKNz7hZ'],
 ['Sigur Rós', 'Untitled #1 - Vaka', 'spotify:track:331TLlSwGbVcNKJMkylNbK'],
 ['Sigur Rós', 'Ekki múkk', 'spotify:track:4aLf8xu5v9TuzVAeIPFETC'],
 ['Sigur Rós',
  'Untitled #4 - Njósnavélin',
  'spotify:track:3fjoFeVDK2ifrZ4yOCwFYC'],
 ['Simon & Garfunkel',
  'Song for the Asking',
  'spotify:track:5t9rkF7g7zIU1NFyrSJaQl'],
 ['Simon & Garfunkel',
  'The Dangling Conversation',
  'spotify:track:1tQy2AqgFc0AdddmIboIfX'],
 ['Stevie Ray Vaughan', 'Little Wing', 'spotify:track:5XNpdKmlLJPUbwKQceX2tW'],
 ['Taylor Swift',
  "All Too Well (10 Minute Version) (Taylor's Version) (From The Vault)",
  'spotify:track:5enxwA8aAbwZbf5qCHORXi'],
 ['Taylor Swift', 'cardigan', 'spotify:track:4R2kfaDFhslZEMJqAFNpdd'],
 ['Taylor Swift',
  'Fearless (Taylor’s Version)',
  'spotify:track:77sMIMlNaSURUAXq5coCxE'],
 ['Taylor Swift', 'willow', 'spotify:track:0lx2cLdOt3piJbcaXIV74f'],
 ['The Avett Brothers',
  'No Hard Feelings',
  'spotify:track:711WfDztCZpnmJg7Uvwod3'],
 ['The Beach Boys',
  'Disney Girls (1957) - Remastered 2009',
  'spotify:track:17qLiavc8woWHAPAGZD0Py'],
 ['The Beach Boys',
  'In My Room - Remastered',
  'spotify:track:62fX8EW16l8St2yL8rMer9'],
 ['The Beach Boys',
  "Surf's Up - Remastered 2009",
  'spotify:track:5YniFjdw9nU8jCzvWlVVQC'],
 ['The Frames', 'Falling Slowly', 'spotify:track:2GsPNtDbbTLnQ0nISq9MmC'],
 ['The Killers',
  "Ruby, Don't Take Your Love To Town",
  'spotify:track:5hg1AkrHaddlxujq6KXvJg'],
 ['The Swell Season',
  'Falling Slowly - Live',
  'spotify:track:2WN4JGHZklVVUki2sJMIHS'],
 ['The Zombies',
  'This Will Be Our Year',
  'spotify:track:560OsnDa7tpsK2B0krVlOP'],
 ['TOOL', 'Lateralus', 'spotify:track:7tvuLLroI0n6uYBWuFig5d'],
 ['Tracy Chapman',
  'Stand by Me - Live at the Late Show with David Letterman',
  'spotify:track:2gs8HVC6KXOQe76XggzZH5'],
 ['Traditional',
  'Fantasia on Greensleeves',
  'spotify:track:6oP2WdkZKffy7lcBQ7lX0o'],
 ['Ultravox', 'Vienna', 'spotify:track:2LXjhNee4iPRVemx9NSZQ3'],
 ['Van Morrison',
  'Madame George - 1999 Remaster',
  'spotify:track:1N4MKISvC1ddfRCRQDXDd2'],
 ['Van Morrison',
  'Slim Slow Slider - 1999 Remaster',
  'spotify:track:35T4VLNlGag05bk0QisZrN'],
 ['Antonio Vivaldi',
  'The Four Seasons - Winter in F Minor, RV. 297: I. Allegro non molto',
  'spotify:track:0ROMalDdNg3L2HoZP4qFyJ'],
 ['Weyes Blood',
  'Do You Need My Love',
  'spotify:track:0lrN5e75X8Pbakrh4E1Ivd'],
 ['Wolfgang Amadeus Mozart',
  'Requiem in D Minor, K. 626: III. Seq. 5, Confutatis',
  'spotify:track:0HsYmC5ahRRjUOfR0cAn0j'],
]


In [101]:
addlist2 = [a[2] for a in addlist]

print (len(addlist2), 'items')

while(addlist2):
    sp.user_playlist_add_tracks(os.getenv('SPOTIFY_USERNAME'), 
                                playlist_id=playlist_id, 
                                tracks=addlist2[-100:])
    addlist2 = addlist2[:-100]
    print("added items, remaining ", len(addlist2))


122 items
Enter the URL you were redirected to: https://druce.ai/?code=AQAPx9KazebpdOMl36sK4To3R2VZdIYqVI6yt_Fe8J_7Ajsldwc7ubIQYMhOP7kJ3aPQ8r5a54KwTJpXFm9gW9-TgumUztlwfoIHN31ISAOGUpUsTvt_ky91ULsG7AO95ceV-XRaK_z29jp9jAT45Feq5fo8EX76OTWTQA0krmdE25DXYBSs4xbk6_A
added items, remaining  22
added items, remaining  0


In [103]:
len(addlist)

203